In [2]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Probability

Let's calculate the probability of getting at least one head out of k coin flips, where k goes from 1 to 50.

In [ ]:
# Create a table of probabilities of getting at least one head








In [ ]:
# let's visualize the results with a scatter plot


## Random Sampling ##

Let's suppose our population is United Airlines domestic flights departing from San Francisco in the summer of 2015; i.e., United Airlines flights departing from San Francisco from 6/1/15 to 8/9/15.

Below we load this population of data and then sample from it using different methods. 


In [3]:
# United Airlines domestic flights departing from San Francisco in the summer of 2015
united = Table.read_table('united_summer2015.csv')
united = united.with_column('Row', np.arange(united.num_rows)).move_to_start('Row')

In [53]:
united

Row,Date,Flight Number,Destination,Delay
0,6/1/15,73,HNL,257
1,6/1/15,217,EWR,28
2,6/1/15,237,STL,-3
3,6/1/15,250,SAN,0
4,6/1/15,267,PHL,64
5,6/1/15,273,SEA,-6
6,6/1/15,278,SEA,-8
7,6/1/15,292,EWR,12
8,6/1/15,300,HNL,20
9,6/1/15,317,IND,-10


In [4]:
# Deterministic sample


In [5]:
# Systematic sample 




In [6]:
# simple random sample





## Probability distributions ##

Let's look at the probability distribution for rolling a fair six-sided die and how the empirical distribution is a good approximation when the sample size gets large. 


In [12]:
# Options for a six-side die
die = Table().with_column('Face', np.arange(1, 7))
die

Face
1
2
3
4
5
6


In [7]:
# Table method .sample() draws at random with replacement 
# from the rows of a table (optional argument: with_replacement=False)


In [8]:
# Setup bins for the die options (middle of bin are integers)



In [9]:
# Theoretical distribution (equal chance for each outcome)


In [10]:
# Empirical distribution based on sample of size 10


## Large Random Samples ##

In [11]:
# Another empirical distribution based on sample of size 10


In [12]:
# Increase sample size to 100


In [14]:
# Another empirical  distribution with a sample size of 100
# These appear more stable than with a sample size of 10


In [13]:
# Another with a sample size of 1000
# More stable than those with a sample size of 10 or 100
# Closer to the theoretical distribution (equal probabilities)


## Sampling from the population of United flight delays

If we view the flight delays of United Airline flights from San Francisco as a population, we can plot the distribution of this population as a histogram. 

If we take large enough samples from this population, the histogram of these samples (empirical distributions) will resemble the population of all flight delays. 


In [24]:
# Let's get back to the United flight delay data. Recall the column headers:
united.show(5)

Row,Date,Flight Number,Destination,Delay
0,6/1/15,73,HNL,257
1,6/1/15,217,EWR,28
2,6/1/15,237,STL,-3
3,6/1/15,250,SAN,0
4,6/1/15,267,PHL,64


In [15]:
# Plot a histogram of the delays. Since we are viewing the delays in the data set as a population, 
# this is the probability distribution of the delays 

# (side note: we will be the data using bins at: np.arange(-20, 201, 10)
# This binning scheme does not display about 0.8% of the data with extreme delays, 
# but makes it easier to see the majority of "shape" of the probability distribution)



In [16]:
# let's draw a sample of size 10 from the population of flight delays and view the corresponding empirical distributions


In [17]:
# Another empirical distribution with a sample size of 100


## Simulating Statistics ##

In [19]:
# Considering united as a population
# The population median delay is:


In [20]:
# Percentage of data less than or equal to the median


In [21]:
# What if we take a random sample of size 30 - what's the estimated median?


In [71]:
# Simulate the empirical sampling distribution of the median (statistic) using a sample size of 30
# We generate 10000 samples of size 30 (there will be 10000 estimates of the median)
# This cell takes a few seconds to run





In [18]:
# Display the empirical distribution of the median as a histogram
